In [107]:
using LinearAlgebra
using GenericLinearAlgebra


##  Objetivo

Dado uma **matriz unitária ou ortogonal** $A \in \mathbb{R}^{(m+n) \times (p+q)}$, queremos fazer a **CS decomposition**:

$$
A = 
\begin{bmatrix}
U_1 & 0 \\
0 & U_2
\end{bmatrix}
\begin{bmatrix}
C & -S & 0 \\
S & C & 0 \\
0 & 0 & I
\end{bmatrix}
\begin{bmatrix}
V_1^\top & 0 \\
0 & V_2^\top
\end{bmatrix}
$$

Esse é o caso mais geral (com bloco identidade no final se sobrarem dimensões), mas vamos focar no **caso simples** para clareza:

### Suponha:

$$
A \in \mathbb{R}^{(m + n) \times r},\quad \text{com } m = n = r
\Rightarrow A \in \mathbb{R}^{2r \times r}
$$

E seja:

$$
A = \begin{bmatrix}
A_1 \\
A_2
\end{bmatrix},\quad A_1, A_2 \in \mathbb{R}^{r \times r}
$$




## Passo a passo da CSD via SVD


### - **Passo 1: Obter os subespaços**

Seja:

* $A_1 \in \mathbb{R}^{r \times r}$
* $A_2 \in \mathbb{R}^{r \times r}$

Você quer decompor $A$ entendendo a **relação geométrica entre os subespaços linha** de $A_1$ e $A_2$.



### - **Passo 2: Aplicar SVD em $A_1$**

Faça:

$$
A_1 = U_1 \Sigma_1 V^\top
$$

* $U_1 \in \mathbb{R}^{r \times r}$ (ortogonal)
* $\Sigma_1 = \text{diag}(\sigma_1^{(1)}, \dots, \sigma_r^{(1)})$
* $V \in \mathbb{R}^{r \times r}$ (ortogonal)


### - **Passo 3: Aplicar SVD em $A_2$** usando **mesmo $V$**

Agora, defina:

$$
\tilde{A}_2 = A_2 V
$$

E aplique SVD em $\tilde{A}_2$:

$$
\tilde{A}_2 = U_2 \Sigma_2
$$

Você agora tem:

* $A_1 = U_1 \Sigma_1 V^\top$
* $A_2 = U_2 \Sigma_2 V^\top$

Essas decomposições **compartilham a mesma base $V$** (isso é importante e possível porque $A$ é unitária).



### - **Passo 4: Obter os ângulos principais**

Agora, queremos achar os ângulos $\theta_i$ tais que:

$$
\cos(\theta_i) = \sigma_i^{(1)}, \quad \sin(\theta_i) = \sigma_i^{(2)}
$$

Como $A$ é unitária:

$$
A_1^\top A_1 + A_2^\top A_2 = I
\Rightarrow \Sigma_1^2 + \Sigma_2^2 = I
$$

Portanto, podemos definir:

$$
\theta_i = \arccos(\sigma_i^{(1)}) = \arcsin(\sigma_i^{(2)})
$$





### - **Passo 5: Construir os blocos da CSD**

* $C = \mathrm{diag}(\cos(\theta_1), \dots, \cos(\theta_r))$
* $S = \mathrm{diag}(\sin(\theta_1), \dots, \sin(\theta_r))$

A matriz do meio da CSD será:

$$
\begin{bmatrix}
C & -S \\
S & C
\end{bmatrix}
$$

E a CSD completa é:

$$
A = 
\begin{bmatrix}
U_1 & 0 \\
0 & U_2
\end{bmatrix}
\begin{bmatrix}
C & -S \\
S & C
\end{bmatrix}
V^\top
$$



##  Recapitulando (Fluxo)

1. **Particione** $A$ em $A_1$ e $A_2$
2. **SVD** de $A_1 = U_1 \Sigma_1 V^\top$
3. Defina $\tilde{A}_2 = A_2 V$, depois faça $\tilde{A}_2 = U_2 \Sigma_2$
4. Calcule $\theta_i = \arccos(\sigma_i^{(1)})$
5. Monte $C, S$, e a decomposição completa



###  Observação importante

Esse processo depende de $A$ ser **unitária/ortogonal** — ou seja, $A^\top A = I$. Caso contrário, essa relação $\Sigma_1^2 + \Sigma_2^2 = I$ **não se sustentaria**, e os cossenos e senos não formariam uma matriz de rotação.



In [108]:
function build_csd_input_matrix(D1::Matrix{Float64}, D2::Matrix{Float64})
    # D1: m × d (m amostras, d features) - dataset 1
    # D2: n × d (n amostras, d features) - dataset 2

    # Passo 1: QR dos transpostos (colunas de D1 e D2 como vetores base)
    Q1, R1 = qr(D1')  
    Q2, R2 = qr(D2') 
    
    
    if size(R1, 1) == size(R1, 2)
        m= size(Q1, 1)
        n = size(R1, 2)
        R1_full = zeros(m, n)
        R1_full[1:n, :] = R1
    else
        R1_full = R1
    end
    if size(R2, 1) == size(R2, 2)
        m= size(Q2, 1)
        n = size(R2, 2)
        R2_full = zeros(m, n)
        R2_full[1:n, :] = R2      
    else
        R2_full = R2  
    end
    

    # Passo 2: Ortonormaliza a união dos subespaços de Q1 e Q2
    Q_all, _ = qr([Q1 Q2])  # Q_all ∈ ℝ^{d × r}, com r ≤ m + n

    # Passo 3: Projeta Q1 e Q2 no subespaço comum Q_all
    A1 = Q_all' * Q1  # ∈ ℝ^{r × m}
    A2 = Q_all' * Q2  # ∈ ℝ^{r × n}
    
    # Passo 4: Constrói a matriz A com blocos
    top = hcat(A1, zeros(size(A1, 1), size(A2, 2)))    # [A1  0]
    bottom = hcat(zeros(size(A2, 1), size(A1, 2)), A2)  # [0   A2]
    A = vcat(top, bottom)  # ∈ ℝ^{2r × (m + n)}

    return A, Q_all, R1_full, R2_full
end

build_csd_input_matrix (generic function with 1 method)

In [109]:
# Exemplo com dados aleatórios
m = 5  # amostras de D1
n = 7  # amostras de D2
d = 6  # número de features

D1 = randn(m, d)  # dataset 1
D2 = randn(n, d)  # dataset 2

A, Q_all, R1, R2 = build_csd_input_matrix(D1, D2)

([1.0 1.1102230246251565e-16 … 0.0 0.0; -5.551115123125783e-17 0.9999999999999998 … 0.0 0.0; … ; 0.0 0.0 … 0.7388686434857469 -0.22241675442881115; 0.0 0.0 … 0.44807017111710873 0.50550251379956], LinearAlgebra.QRCompactWYQ{Float64, Matrix{Float64}, Matrix{Float64}}([0.9999999999999999 1.1102230246251565e-16 … 0.4408423883505542 -0.26452958949013616; -0.12651859192983814 0.9999999999999999 … -0.19170625465820368 0.2765499507421787; … ; -0.390608965664288 -0.11642669828652653 … 0.7388686434857469 -0.22241675442881115; -0.3200151963255692 -0.26123881300671764 … 0.44807017111710873 0.50550251379956], [1.3989700954632844 -0.29377669446131993 … 0.36425808070105953 0.0; 6.3659873764e-314 1.2935777460502251 … 1.4255864467407124 0.0; … ; 2.33419537006e-313 3.60739284543e-313 … 1.1790064876837345 0.0; 1.9097962123e-313 5.0927898992e-313 … 0.0 0.0]), [-3.6166112707633324 1.415567587796891 … 0.6172484906896993 0.7852835843839266; 0.0 2.696327999681108 … -1.675154980222291 0.04098385814361112; … ;

In [110]:
# Exibir a matriz A
display(A)

12×12 Matrix{Float64}:
  1.0           1.11022e-16  …   0.0        0.0         0.0
 -5.55112e-17   1.0              0.0        0.0         0.0
  0.0           5.55112e-17      0.0        0.0         0.0
 -5.55112e-17   1.11022e-16      0.0        0.0         0.0
  1.11022e-16  -9.71445e-17      0.0        0.0         0.0
  1.11022e-16  -2.77556e-17  …   0.0        0.0         0.0
  0.0           0.0              0.167951   0.440842   -0.26453
  0.0           0.0             -0.648237  -0.191706    0.27655
  0.0           0.0             -0.277996  -0.0732448  -0.207081
  0.0           0.0              0.513057   0.1298      0.711097
  0.0           0.0          …  -0.116601   0.738869   -0.222417
  0.0           0.0             -0.444384   0.44807     0.505503

In [116]:
if A'*A ≈ I
    println("A matriz A é ortonormal")
else
    println("A matriz A não é ortonormal")
    
end

A matriz A é ortonormal


In [111]:
function rebuild_datasets_matrixes(A, Q_all, R1, R2)
    # A: matriz de entrada
    # Q_all: base ortonormal comum
    # R1: matriz triangular superior de D1
    # R2: matriz triangular superior de D2

    r = size(Q_all, 2)  # dimensão do subespaço comum

    A1 = A[1:r, 1:size(R1, 1)] 
    D1_reconstructed = (Q_all * (A1' \ R1))'
   
    A2 = A[r+1:end, end-size(R2,1)+1:end] 
    D2_reconstructed = (Q_all * (A2' \ R2))'
    
    # D1_reconstructed: matriz reconstruída de D1
    # D2_reconstructed: matriz reconstruída de D2

    # Retorna as matrizes reconstruídas
    return D1_reconstructed, D2_reconstructed
    
end

rebuild_datasets_matrixes (generic function with 1 method)

In [ ]:
D1_reconstructed, D2_reconstructed = rebuild_datasets_matrices(A, Q_all, R1, R2)

if D1_reconstructed ≈ D1 && D2_reconstructed ≈ D2
    println("As matrizes reconstruídas são equivalentes às originais!")
else
    println("As matrizes reconstruídas NÃO são equivalentes às originais.")
    
end

As matrizes reconstruídas são equivalentes às originais!
